# A1196 Conservation Analysis
*Elliot Williams*

*August 13th, 2019*


**Note to users of this code**: Let me know! I would definitely be interested in whatever it is being used for.
Also, if you publish anything using it, please cite me!


Question: **Is A1196/A1427 (*S. cerevisiae* / *E. coli* numbering) of the 18S/16S rRNA well conserved within RSCB deposited ribosome-structures? Is this also true of C1054?**

## Background

The Weir Lab has been conducting MD simulations of the region of the ribosome within a 35 angstrom sphere surrounding G530 of the 16S/18S rRNA. The structures for these simulations were obtained from the five cryo-EM translocation intermediate structures obtained from [this paper](https://elifesciences.org/articles/14874), out of the Korostelev research group out of UMass Medical School.

Within these simulations (presumably), it has been noticed that A1196 (which base stacks with C1054) transiently interacts via Hoogsteen/Watson-Crick base pairing with the second nucleotide of the incoming A-site codon, raising the question of a model in which the codon:anticodon base pairing is extended to 5 base pairs.

The goals of this assessment is to find out whether or not the A1427 residue is well-conserved in the [RCSB](http://www.rcsb.org/)-deposited ribosome structures available. Additionally, as C1054 has been shown to [interact with the mRNA](https://academic.oup.com/nar/article/44/13/6036/2457638) during molecular dynamics simulations (as well as during our own Korostelev structure simulations), it would be interesting to look at conservation of this base within structures as well.

One would expect that conservation of a residue, either on an intraspecies or interspecies level, would suggest mechanistic importance of the residue to the ribosome's function; so, getting a sense of a particular residue's level of conservation will allow us, by proxy, to get at a residue's functional importance.

If we find intraspecies conservation but not interspecies conservation, we might expect that the residue is important for the ribosome's function at a species-specific level. If no conservation is found at all, it is unlikely that the residue has high functional relevance.

With all of this background stated before I taint my thoughts with the analysis at hand, let us move forward to the task at hand.




## Data Collection

### Data Source Description

The data I am collecting to conduct my analysis is sourced from the [RCSB](http://www.rcsb.org/), an online database of Protein Data Bank (PDB) files representing the 3D shapes of proteins, nucleic acids, and complex assemblies.

### RCSB API implementation

I'm reusing a RCSB API implementation I made last year to fetch data from the website, defined within `rcsb_api.py`.

The benefit of this is that I can get any data I please, if I implement the API logic.
The drawback is that any new queries must be implemented. 

See [PyPDB](http://www.wgilpin.com/pypdb_docs/html/) for a more 
mature project with more limited functionality.

In [1]:
# All dependencies, including the rcsb_api.py dependencies (of `requests`, `lxml`, `pandas`),
# should be installed for this to work properly.

# First, let's import the RCSB Python API I defined
import rcsb_api as rcsb
import pandas as pd

# And let's set the pandas display variables
pd.set_option('display.max_colwidth', 1000)
pd.set_option('display.max_rows', 50)

In [2]:
# Then, let's query for all PDB IDs which have 'ribosome' in their title
pdb_ids = rcsb.query_for_pdb_ids(queryTitle="ribosome", verbose=False)
# And let's get all associated metadata with these IDs so we can see 
pdb_ids_w_info = rcsb.get_info_from_ids(pdb_ids)

# # The following lines print the DataFrame in full
# with pd.option_context('display.max_rows', None, 
#                        'display.max_columns', None,
#                        'display.max_colwidth', None):
#     display(pdb_ids_w_info)

# I don't want to keep scrolling past this, so let's actually only print some rows
display(pdb_ids_w_info)

,structureId,structureTitle,resolution,depositionDate,experimentalTechnique,residueCount
0,1AHA,THE N-GLYCOSIDASE MECHANISM OF RIBOSOME-INACTIVATING PROTEINS IMPLIED BY CRYSTAL STRUCTURES OF ALPHA-MOMORCHARIN,2.20,1994-01-07,X-RAY DIFFRACTION,246
1,1AHB,THE N-GLYCOSIDASE MECHANISM OF RIBOSOME-INACTIVATING PROTEINS IMPLIED BY CRYSTAL STRUCTURES OF ALPHA-MOMORCHARIN,2.20,1994-01-07,X-RAY DIFFRACTION,246
2,1AHC,THE N-GLYCOSIDASE MECHANISM OF RIBOSOME-INACTIVATING PROTEINS IMPLIED BY CRYSTAL STRUCTURES OF ALPHA-MOMORCHARIN,2.00,1994-01-07,X-RAY DIFFRACTION,246
3,1DD5,"CRYSTAL STRUCTURE OF THERMOTOGA MARITIMA RIBOSOME RECYCLING FACTOR, RRF",2.55,1999-11-08,X-RAY DIFFRACTION,185
4,1DK1,DETAILED VIEW OF A KEY ELEMENT OF THE RIBOSOME ASSEMBLY: CRYSTAL STRUCTURE OF THE S15-RRNA COMPLEX,2.80,1999-12-06,X-RAY DIFFRACTION,143
...,...,...,...,...,...,...
741,6S0X,Erythromycin Resistant Staphylococcus aureus 70S ribosome (delta R88 A89 uL22) in complex with erythromycin.,2.42,2019-06-18,ELECTRON MICROSCOPY,10148
742,6S0Z,Erythromycin Resistant Staphylococcus aureus 50S ribosome (delta R88 A89 uL22) in complex with erythromycin.,2.30,2019-06-18,ELECTRON MICROSCOPY,6125
743,6S12,Erythromycin Resistant Staphylococcus aureus 50S ribosome (delta R88 A89 uL22).,3.20,2019-06-18,ELECTRON MICROSCOPY,6145
744,6S13,Erythromycin Resistant Staphylococcus aureus 70S ribosome (delta R88 A89 uL22).,3.58,2019-06-18,ELECTRON MICROSCOPY,10167


## PDB ID Filtering

It is obvious if you look through the titles that only some of these structures are even of the ribosome, let alone structures containing the 16S/18S rRNA. So, let's try to get a list of what elements these structures contain, such that we can obtain only the structures we want, we need to apply a filter.

We only want structures with the 16S/18S rRNA, so let's get the descriptions of the chains of each structure,
and use it as a filtering condition.

As an example, let's see what the chain descriptions of one structure look like:

In [3]:
rcsb.describe_chains("6S0X").Description

0                         23S ribosomal RNA
1                 50S ribosomal protein L15
2                 50S ribosomal protein L16
3                 50S ribosomal protein L17
4                 50S ribosomal protein L18
                      ...                  
47    Ribosome hibernation promoting factor
48                 50S ribosomal protein L5
49                 50S ribosomal protein L6
50                50S ribosomal protein L13
51                50S ribosomal protein L14
Name: Description, Length: 52, dtype: object

Let's collect this information for all of our PDB IDs,
and try to see how many possible chain names there are.

In [4]:
# Sets containing all unique chain names and descriptions in all structures
chain_name_set = set()
chain_description_set = set()

for (index, pdb_id) in enumerate(pdb_ids):
    # Prints out string to update on progress
    if index % 50 == 0:
        print("Getting chains for %s (%d/%d)" % (pdb_id, index+1, len(pdb_ids)))
    
    # Attempts to get chain descriptions from the API 
    # (HTTP request could fail due to flakiness; will print out if so)
    try:
        chain_df = rcsb.describe_chains(pdb_id)
        for chain_name in chain_df.Name:
            chain_name_set.add(chain_name)
        for chain_description in chain_df.Description:
            chain_description_set.add(chain_description)
    except:
        print("Failed to get chains for structure %s" % pdb_id)

Getting chains for 1AHA (1/746)
Getting chains for 1RY1 (51/746)
Getting chains for 2QES (101/746)
Getting chains for 3JAJ (151/746)
Getting chains for 4CSU (201/746)
Getting chains for 4TUE (251/746)
Getting chains for 4V50 (301/746)
Getting chains for 4V7E (351/746)
Getting chains for 4W29 (401/746)
Getting chains for 5BY8 (451/746)
Getting chains for 5J88 (501/746)
Getting chains for 5NDJ (551/746)
Getting chains for 5WFS (601/746)
Getting chains for 6FKR (651/746)
Getting chains for 6MTC (701/746)


In [5]:
# Let's see how many unique chain names and chain descriptions there are
print("There are %d unique chain names" % len(chain_name_set))
print("There are %d unique chain descriptions" % len(chain_description_set))

There are 1024 unique chain names
There are 3291 unique chain descriptions


In [6]:
# What do chain names tend to look like?
display(list(chain_name_set)[1:20])

['40S ribosomal protein S10, putative',
 '50S ribosomal protein L15, chloroplastic',
 '28S ribosomal protein S10, mitochondrial',
 '60S ribosomal subunit protein L31, putative',
 'Ribosomal protein L17',
 '37S ribosomal protein S28, mitochondrial',
 'Apolipoprotein A-I',
 '50S ribosomal protein L18, chloroplastic',
 '40S ribosomal protein S14',
 'Putative U3 snoRNP protein',
 'Ribosomal protein L34a, isoform C',
 '30S ribosomal protein S13',
 'Signal recognition particle receptor subunit beta',
 '60S ribosomal protein L31',
 'Mitochondrial ribosomal protein S11',
 '40S ribosomal protein S14b',
 'Mitoribosomal protein bs18m, mrps18c',
 '40S ribosomal protein S28-B',
 'Beta-galactoside-specific lectin 4']

In [7]:
# What do chain descriptions tend to look like?
display(list(chain_description_set)[1:20])

['GUANINE NUCLEOTIDE-BINDING PROTEIN SUBUNIT BETA-2-LIKE 1',
 '40S RIBOSOMAL PROTEIN S21, PUTATIVE',
 'Apolipoprotein A-I',
 '40S ribosomal protein rpS30 (S30e)',
 'plastid ribosomal protein uL5c',
 '40S ribosomal protein S7-like protein',
 'MITORIBOSOMAL PROTEIN ML42, MRPL42',
 'Mitochondrial 16S ribosomal RNA',
 'ribosomal protein eS4',
 'eS31',
 '40S ribosomal protein S28-B',
 'uS5 (yeast S2)',
 '60S ribosomal protein L43E',
 'Selenocysteine-specific elongation factor',
 '39S ribosomal protein L44, mitochondrial',
 '40S ribosomal protein S15',
 'Ribosomal protein s1e',
 'A-site tRNA',
 'uS11 (yeast S14)']

It seems at first glance that chain names and chain descriptions are used mostly interchangeably. 

In [8]:
# Let's try printing all chain names with "16" or "18" in their names,
# to give us an idea of what kind of filters would work to get all of the 16S/18S rRNA Structures
# (Note: For display purposes, only printing first 10 names)
display(list(filter(lambda x: ("16" in x) or ("18" in x), chain_name_set ))[0:10] + ["etc..."])

['Mitoribosomal protein bs18m, mrps18c',
 'Tetracycline resistance protein TetM from transposon Tn916',
 'Ribosomal protein S16',
 '40S ribosomal protein S16, putative',
 '28S ribosomal protein S18c, mitochondrial',
 '30S ribosomal protein S18, chloroplastic',
 '50S ribosomal protein L18Ae',
 '60S ribosomal protein L11 (L5, L16)',
 '30S ribosomal protein S16, chloroplastic',
 'etc...']

In [9]:
# Do the same for chain descriptions
# (Note: For display purposes, only printing first 10 descriptions)
display(list(filter(lambda x: ("16" in x) or ("18" in x), chain_description_set ))[0:10]+ ["etc..."])

['50S Ribosomal Protein L16',
 '18S RIBOSOMAL RNA',
 '40S ribosomal protein rpS16 (S9p)',
 'MRPL18',
 '16S RIBOSOMAL RNA',
 '50S ribosomal protein L10E/L16',
 '30S Ribosomal Protein S18',
 'Ribosomal protein S16',
 '40S ribosomal protein S16(A)',
 'etc...']

Having looked through all of these chain names and decriptions, I'm officially convinced that the vast majority of 16S/18S rRNA candidates will have `16S` or `18S` in the their descriptions

So, let's filter our set of structures to just contain those that have 16S or 18S in their descriptions.

In [10]:
# This function returns a bool representing whether or not the
# PDB structure has any of the given substrings in any of its chain names,
# chain descriptions, or both (depending on value of query_fields).
#
# Input:
#   pdb_id : PDB ID of structure we want to evaluate
#   substring_list : List of substrings to query on
#   query_fields : Which pdb chain column(s) we want to filter on
#                  (must be sublist of ["Name", "Description"])
#   num_attempts : The number of times to try the HTTP request to the RCSB API before
#                  giving up (and returning False)
# 
# Output:
#   Whether or not the PDB structure contains any chains satisfying these conditions.
# 
# EG  `chainFieldsContainSubstrings(["16S", "18S"], "5JUP", ["Description"])`
# should return `True`, as it has 18S chain description
# 
# And `chainFieldsContainSubstrings(["16S", "18S"], "5JUP", ["Name", "Description"])`
# should return `True`, as it has 18S chain description, but no 18S chain name
# 
# But `chainFieldsContainSubstrings(["16S", "18S"], "5JUP", ["Name"])`
# should return `False`, as it has no 18S chain description
def chainFieldsContainSubstrings(pdb_id, substring_list, 
                                 query_fields=["Name", "Description"],
                                 num_attempts=3):    
    # Retries the RCSB query multiple times (to reduce flakiness)
    num_attempted = 0
    success = False
    while (num_attempted < num_attempts):
        try:
            chain_df = rcsb.describe_chains(pdb_id)
            success = True
            break
        except:
            num_attempted += 1
    if not success:
        print("Failed to obtain chains from '%s'" % pdb_id)
        return False
    
    # Returns `True` if any of the substrings are in any of the chain names
    if "Name" in query_fields:
        for chain_name in chain_df.Name:
            for substring in substring_list:
                if substring in chain_name:
                    return True
    # Returns `True` if any of the substrings are in any of the chain descriptions
    if "Description" in query_fields:
        for chain_description in chain_df.Description:
            for substring in substring_list:
                if substring in chain_description:
                    return True
                
    # Otherwise, return `False`
    return False

In [14]:
# Let's test this function works properly
(chainFieldsContainSubstrings("5JUP", ["16S","18S"], ["Name", "Description"]) and
 not chainFieldsContainSubstrings("5JUP", ["16S","18S"], ["Name"]) and
 chainFieldsContainSubstrings("5JUP", ["16S","18S"], ["Description"]))

True

Now I'm convinced the function actually works as it is supposed to, let's apply
the filter to our PDB IDs, such that we only have IDs with 16S or 18S.

In [12]:
desired_pdb_ids = []
for (index, pdb_id) in enumerate(pdb_ids):
    # Prints out string to update on progress
    if index % 50 == 0:
        print("Filtering structure ''%s' (%d/%d)" % (pdb_id, index+1, len(pdb_ids)))

    if chainFieldsContainSubstrings(pdb_id, substring_list=["16S", "18S"],
                                    query_fields=["Description"]):
        desired_pdb_ids.append(pdb_id)

Filtering structure ''1AHA' (1/746)
Filtering structure ''1RY1' (51/746)
Filtering structure ''2QES' (101/746)
Filtering structure ''3JAJ' (151/746)
Filtering structure ''4CSU' (201/746)
Filtering structure ''4TUE' (251/746)
Filtering structure ''4V50' (301/746)
Filtering structure ''4V7E' (351/746)
Filtering structure ''4W29' (401/746)
Filtering structure ''5BY8' (451/746)
Filtering structure ''5J88' (501/746)
Filtering structure ''5NDJ' (551/746)
Filtering structure ''5WFS' (601/746)
Filtering structure ''6FKR' (651/746)
Filtering structure ''6MTC' (701/746)
['1EG0', '1JGO', '1JGP', '1JGQ', '1MVR', '1QZC', '1T1M', '1T1O', '1TRJ', '1VVJ', '1VY4', '1VY5', '1VY6', '1VY7', '1ZC8', '2F4V', '2FTC', '2NOQ', '2OM7', '3J0D', '3J0E', '3J16', '3J5S', '3J6X', '3J6Y', '3J77', '3J78', '3J7A', '3J7P', '3J7R', '3J9M', '3J9W', '3J9Y', '3JAJ', '3JAN', '3JBN', '3JBO', '3JBP', '3JBU', '3JCD', '3JCE', '3JCJ', '3JCN', '486D', '4AQY', '4B3M', '4B3R', '4B3S', '4B3T', '4CE4', '4CXG', '4CXH', '4L47', '4L71', 

In [13]:
print(len(desired_pdb_ids))

458


In [24]:
# Let's see how many 16S/18S rRNA candidates we have in all those structures
# (and which structures, if any, have repeats)
chain_list = []
id_seen_once = set()
id_seen_twice = set()

for (index, pdb_id) in enumerate(desired_pdb_ids):
    # Prints out string to update on progress
    if index % 50 == 0:
        print("Getting chains for %s (%d/%d)" % (pdb_id, index+1, len(desired_pdb_ids)))
    
    # Attempts to get chain descriptions from the API 
    # (HTTP request could fail due to flakiness; will print out if so)
    try:
        chain_df = rcsb.describe_chains(pdb_id)
        for chain_description in chain_df.Description:
            if "16S" in chain_description or "18S" in chain_description:
                chain_list.append((pdb_id, chain_description))
                if pdb_id not in id_seen_once:
                    id_seen_once.add(pdb_id)
                else:
                    id_seen_twice.add(pdb_id)
    except:
        print("Failed to get chains for structure %s" % pdb_id)

Getting chains for 1EG0 (1/458)
Getting chains for 4CXG (51/458)
Getting chains for 4V4I (101/458)
Getting chains for 4V6I (151/458)
Getting chains for 4V9K (201/458)
Getting chains for 5E81 (251/458)
Getting chains for 5LL6 (301/458)
Getting chains for 5V93 (351/458)
Getting chains for 6GXP (401/458)
Getting chains for 6QZP (451/458)


In [28]:
print(len(chain_list))

458


Notably, 507 != 458. So, let's see which structures have repeats.

In [26]:
print(len(id_seen_once))
print(list(id_seen_twice))
print(len(id_seen_twice))

458
['6BOK', '4WR6', '6HCM', '4CXG', '6OLF', '5J4D', '1ZC8', '4V6D', '1TRJ', '6OLI', '4V6F', '486D', '4V7T', '5MYJ', '6I7O', '6OM0', '2OM7', '3J0E', '4V7V', '3J0D', '4V88', '6B4V', '5W4K', '6BOH', '4V7S', '4CXH', '6OM7', '1MVR', '4V6C', '4V6E', '6HCF', '4WU1', '6OLE', '4WQ1', '2NOQ']
35


After a brief look at the FASTA files of those structures, it seems that they mostly have repeated 16S rRNA chains with identical sequences. We'll treat this more rigorously later.

## Afterword: My Analysis Technique

The layout of this notebook was inspired by [this paper](https://arxiv.org/abs/1901.08152) about formatting notebook formats to maximize reproduciblity of data science analyses, drawing from the principles of predictability, computatibility, and stability. 

IE if someone else did this analysis, if I did my job right, they should always get the same conclusions as they will always know which judgement calls I made over the course of my analysis.